In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    setting = yaml.load(yaml_file)

path = make_path_dict(setting)

In [ ]:
if setting["signal_type"] == "raw":

    raise

In [ ]:
feature_x_sample_processed = pd.read_table(
    path["feature_x_sample.processed.tsv"], index_col=0
)

In [ ]:
for element_type, df in (
    ("feature", feature_x_sample_processed),
    ("sample", feature_x_sample_processed.T),
):

    element_x_fit_parameter_file_path = path[
        "{}_x_fit_parameter.tsv".format(element_type)
    ]

    element_x_fit_parameter = ccal.fit_skew_t_pdfs(
        df,
        n_job=setting["max_n_job"],
        output_file_path=element_x_fit_parameter_file_path,
    )

    element_x_fit_parameter = pd.read_table(
        element_x_fit_parameter_file_path, index_col=0
    )

    element_alias = setting["{}_alias".format(element_type)]

    for parameter, parameter_values in element_x_fit_parameter.items():

        if parameter_values.unique().size < 2:

            continue

        ccal.plot_distributions(
            (parameter_values,),
            names=(parameter,),
            plot_rug=parameter_values.size < setting["plot_rug_max_size"],
            title="{}<br>Distribution of {}".format(element_alias, parameter),
            xaxis_title=parameter,
        )

        parameter_values_sorted = parameter_values.sort_values()

        first_element = parameter_values_sorted.index[0]

        last_element = parameter_values_sorted.index[-1]

        ccal.plot_distributions(
            (df.loc[first_element], df.loc[last_element]),
            names=(first_element, last_element),
            plot_rug=df.shape[1] < setting["plot_rug_max_size"],
            title="{}<br>Distributions with Extreme {}".format(
                element_alias, parameter
            ),
            xaxis_title=setting["feature_x_sample_value_name"],
        )

        n_extreme = 3

        for rank in list(range(n_extreme)) + list(range(-n_extreme, 0)):

            ranked_element = parameter_values_sorted.index[rank]

            element_values = df.loc[ranked_element]

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                ranked_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            ccal.plot_context(
                element_values,
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=element_values.dropna().size < setting["plot_rug_max_size"],
                title="{} {} Rank {}: {}".format(
                    element_alias, parameter, rank, ranked_element
                ),
                xaxis_title=setting["feature_x_sample_value_name"],
            )